In [36]:
import constant, tensor, gate, utilities
import numpy as np, qiskit, re

def qasm_to_qasmgates(qasm):
    gates = qasm.split('\n')[3:-1]
    qasm_gates = []
    for gate in gates:
        gate = gate[:-1].split(' ')
        # print(gate)
        indices = re.findall(r'\d+', gate[1])
        indices = [int(index) for index in indices]
        matches = re.match(r'([a-z]+)(\(\d+\.\d+\))?', gate[0])
        # Extract the function name and value from the matches
        name = matches.group(1).upper()
        if matches.group(2) is None:
            param = 0
        else:
            param = float(matches.group(2)[1:-1])
        qasm_gates.append((name, param, indices))
    return qasm_gates

def qasmgates_to_qcs(gates: list) -> list[qiskit.QuantumCircuit]: 
    qcs = []
    sub_qc = []
    counter = 0
    active_2qubit = 0 # 0 mean dactive, 1 mean active, 2 mean break instruction
    num_qubits = max(max(gates, key=lambda x: max(x[2]))[2]) + 1
    print(num_qubits)
    slots = np.zeros(num_qubits)
    for name, param, indices in gates: 
        if len(indices) == 2:
            active_2qubit += 1
        
        slots = utilities.update_slot(slots, indices)
        if any(slot > 1 for slot in slots) or active_2qubit == 2:
            qcs.append(sub_qc)
            active_2qubit = 0
            sub_qc = []
            sub_qc.append((name, param, indices))
            if len(indices) == 2:
                active_2qubit += 1
            slots = np.zeros(num_qubits)
            slots = utilities.update_slot(slots, indices)
            gates = gates[counter + 1:]
            counter = 0
        else:
            sub_qc.append((name, param, indices))
            counter += 1
            if counter >= len(gates):
                qcs.append(sub_qc)
                return qcs
    return qcs


def qasmgates_to_gates(qasmgates: list[str]):
    gates = []
    for name, param, indices in qasmgates: 
        gates.append(gate.Gate(name, indices = indices, param = param))
    return gates
    
def gates_to_string(gates: list[gate.Gate], num_qubits: int) -> np.ndarray:
    """Return the matrix representation of a circuit composed of a list of gates acting on `num_qubits` qubits.

    Args:
        gates (list[Gate]): List of gates in the circuit
        num_qubits (int): # of qubits in the system

    Returns:
        np.ndarray: Matrix representation of the circuit
    """
    # Example: {CNOT_1,3, RY_2(\pi)} act on 5 qubits circuit
    # Init tensor form as : I \otimes I \otimes ... \otimes I (num_qubits otimes)
    # and params form as [0, 0, ..., 0] (num_qubits times)
    tensor_form = [['I'] * num_qubits]
    params_form = [0] * num_qubits
    # Update tensor form and params form based on gates
    # => tensor_form = [['I', 'M', 'RY', 'I', 'I'], ['I', 'P', 'RY', 'X', 'I']]
    # => params_form = [0, 0, np.pi, 0, 0]
    for gate in gates:
        if gate.param is not None:
            if len(gate.indices) == 1:
                params_form[gate.indices[0]] = gate.param
            else:
                params_form[gate.indices[1]] = gate.param
        if len(gate.indices) == 1:
            tensor_form[0][gate.indices[0]] = gate.name
        else:
            
            tensor_form = utilities.duplicate_xss(tensor_form)
            tensor_form[0][gate.indices[0]] = 'M' # P0
            tensor_form[1][gate.indices[0]] = 'P' # P3
            tensor_form[1][gate.indices[1]] = gate.name
    
    return params_form, tensor_form

def string_to_matrix(params_form: np.ndarray, tensor_form: np.ndarray) -> np.ndarray:
    """Convert the tensor form and params form to the matrix representation of the circuit.

    Args:
        params_form (np.ndarray): lisr of parameters
        tensor_form (np.ndarray): list of gates (only gate names, no parameters)

    Returns:
        np.ndarray: matrix representation of the circuit
    """
    # Convert tensor form and params form to matrix
    # => [['I', 'M', 'RY', 'I', 'I'], ['I', 'P', 'RY', 'X', 'I']]
    # -> I \otimes M \otimes RY(np.pi) \otimes I \otimes I
    #  + I \otimes P \otimes RY(np.pi) \otimes X \otimes I
    matrices = [np.array([1])]*len(tensor_form)
    for i in range(0, len(tensor_form[0])):
        for j in range(len(tensor_form)):
            #print(f'A{tensor_form[j][i]}')
            if len(tensor_form[j][i]) >= 2:
                matrices[j] = getattr(tensor, f'A{tensor_form[j][i]}')(matrices[j], params_form[i])
            else:
                matrices[j] = getattr(tensor, f'A{tensor_form[j][i]}')(matrices[j])
    return sum(matrices)

crx13 = gate.Gate('RX', indices = [1, 3], param = np.pi)
ry2 = gate.Gate('RY', indices = 2, param = np.pi)
params_form, tensor_form = gates_to_string([crx13, ry2], 5)
matrix = string_to_matrix(params_form, tensor_form)
print(matrix.shape)

(32, 32)


In [35]:
import qiskit.quantum_info as qi
import constant
from qoop.core.random_circuit import generate_with_pool
from qoop.core.gradient import grad_loss
import splitter
%load_ext autoreload
%autoreload 2
num_qubits = 5
qc = generate_with_pool(num_qubits, 500)
qc = qc.assign_parameters([1] * qc.num_parameters)
state = qi.Statevector.from_instruction(qc)
# grad_loss1 = grad_loss(qc, [1] * qc.num_parameters)
# print(grad_loss1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
from qoop.core import ansatz
qc = ansatz.Walltoall_zxz(4, 200)
qc = qc.assign_parameters([1] * qc.num_parameters)
state = qi.Statevector.from_instruction(qc)

In [61]:
class U():
    def __init__(self, params_form, tensor_form, index = 0):
        self.params_form = params_form
        self.tensor_form = tensor_form
        self.matrix_form = None
        self.index = index
    def compare(self, us):
        for u in us:
            if self.params_form == u.params_form and self.tensor_form == u.tensor_form:
                self.index = u.index
                self.matrix_form = u.matrix_form
                return True
        return False
    def to_matrix(self):
        if self.matrix_form is None:
            self.matrix_form = string_to_matrix(self.params_form, self.tensor_form)
        return self.matrix_form

matrices = []
Us = []
index = 0
for sub_qc in qcs:
    gates = circuit_to_gates(sub_qc)
    params_form, tensor_form = gates_to_string(gates, num_qubits)
    u = U(params_form, tensor_form, index)
    if u.compare(Us) == False:
        index += 1
    Us.append(u)
    # print(tensor_form)
    matrices.append(u.to_matrix())
result = matrices[0]
for i in range(1, len(matrices)):
    # print(matrices[i].shape)
    result = result @ matrices[i]
# return result
# matrix = qimax(qc)
test_state = result @ constant.state0(num_qubits)

/var/folders/bw/2cm4rnhn0djgg95n4py1pysc0000gp/T/ipykernel_39285/3853086874.py:7: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubit_indices = [q.index for q in qargs]


In [ ]:
k1 = np.kron(np.kron(gate.gate['RX'](1), gate.gate['H']), gate.gate['RX'](1))
k2 = np.kron(np.kron(constant.P(0), gate.gate['RY'](1)), gate.gate['I']) + np.kron(np.kron(constant.P(3), gate.gate['RY'](1)), gate.gate['RX'](1))
print(k2 @ (k1 @ constant.state0(3)))

[ 0.21682799+0.j          0.        -0.11845367j  0.73899825+0.j
  0.        -0.40371659j  0.        -0.07292851j -0.11357943+0.j
  0.        -0.24855667j -0.38710408+0.j        ]


In [ ]:
matrices[1] @ (matrices[0] @ constant.state0(3))

array([ 0.21682799+0.j        ,  0.        -0.11845367j,
        0.73899825+0.j        ,  0.        -0.40371659j,
        0.        -0.10395288j, -0.05678972+0.j        ,
        0.        -0.35429464j, -0.19355204+0.j        ])

In [ ]:
true_state

array([ 0.21682799+0.j        ,  0.        -0.07292851j,
        0.73899825+0.j        ,  0.        -0.24855667j,
        0.        -0.11845367j, -0.11357943+0.j        ,
        0.        -0.40371659j, -0.38710408+0.j        ])